In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import datetime

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
ds = pd.read_csv('C:/Users/hamuj/Desktop/ml/Class/LSTM/Crowdbabble_Social-Media-Analytics_Twitter-Download_Donald-Trump_7375-Tweets.csv', index_col=1, encoding = 'ISO-8859–1')
ds.Tweet_Text.head()

Time
15:26:37    Today we express our deepest gratitude to all ...
13:33:35    Busy day planned in New York. Will soon be mak...
11:14:20    Love the fact that the small groups of protest...
2:19:44     Just had a very open and successful presidenti...
2:10:46     A fantastic day in D.C. Met with President Oba...
Name: Tweet_Text, dtype: object

In [3]:
# Take all the text together

data = ' '.join([ix for ix in ds.Tweet_Text])
print (data[:1000])

Today we express our deepest gratitude to all those who have served in our armed forces. #ThankAVet https://t.co/wPk7QWpK8Z Busy day planned in New York. Will soon be making some very important decisions on the people who will be running our government! Love the fact that the small groups of protesters last night have passion for our great country. We will all come together and be proud! Just had a very open and successful presidential election. Now professional protesters, incited by the media, are protesting. Very unfair! A fantastic day in D.C. Met with President Obama for first time. Really good meeting, great chemistry. Melania liked Mrs. O a lot! Happy 241st birthday to the U.S. Marine Corps! Thank you for your service!! https://t.co/Lz2dhrXzo4 Such a beautiful and important evening! The forgotten man and woman will never be forgotten again. We will all come together as never before Watching the returns at 9:45pm. 
#ElectionNight #MAGA_Ùà¼_Ùàü https://t.co/HfuJeRZbod RT @IvankaTr

In [4]:
print (set(data))
print (len(set(data)))

{')', 'Î', 'ö', 'm', 'r', '"', '=', 'y', ':', ',', '\x99', '\x8f', 'Ð', 'c', '1', 'B', 'w', 'Ó', 'h', 'Ò', 'Â', '¼', '\xa0', '¥', 'O', '´', ' ', 'Ô', 'i', '#', '_', 'T', 'K', 'b', 'Y', '5', '÷', 'Z', 'U', '¬', 'â', '|', '2', 'o', 'F', '«', 'Ù', '3', 'j', 'S', '\x81', '[', 'f', 'z', '?', 'Ñ', 'E', '&', 'X', '~', '8', 'Û', '*', 'ü', 'd', 'e', '»', '4', 'Ä', 'x', '¡', 'R', '@', 'G', 'A', '!', 'Á', '\n', '%', 'Õ', 'p', 'M', 'I', 'g', '9', '\x8d', 'À', 'Ü', 'N', 'Ê', 'ª', 'É', '.', 'å', 'ä', '\x80', '7', 'a', '$', 'C', '-', '(', 'È', 'L', 'à', 'H', '\x8b', '{', ']', 'ø', '£', 'J', 'Q', '0', '6', 't', 'Ï', '¢', 'Ì', '}', '/', 'u', '\x9d', '\x95', 'n', 'W', ';', 'D', '±', 'V', 'l', 'k', '\x89', 's', 'q', '+', 'v', 'P'}
138


In [5]:
# Create Vocab
vocab = list(set(data))

print(vocab)

i2c, c2i = {}, {}

for idx, chx in enumerate(vocab):
    i2c[idx] = chx
    c2i[chx] = idx

[')', 'Î', 'ö', 'm', 'r', '"', '=', 'y', ':', ',', '\x99', '\x8f', 'Ð', 'c', '1', 'B', 'w', 'Ó', 'h', 'Ò', 'Â', '¼', '\xa0', '¥', 'O', '´', ' ', 'Ô', 'i', '#', '_', 'T', 'K', 'b', 'Y', '5', '÷', 'Z', 'U', '¬', 'â', '|', '2', 'o', 'F', '«', 'Ù', '3', 'j', 'S', '\x81', '[', 'f', 'z', '?', 'Ñ', 'E', '&', 'X', '~', '8', 'Û', '*', 'ü', 'd', 'e', '»', '4', 'Ä', 'x', '¡', 'R', '@', 'G', 'A', '!', 'Á', '\n', '%', 'Õ', 'p', 'M', 'I', 'g', '9', '\x8d', 'À', 'Ü', 'N', 'Ê', 'ª', 'É', '.', 'å', 'ä', '\x80', '7', 'a', '$', 'C', '-', '(', 'È', 'L', 'à', 'H', '\x8b', '{', ']', 'ø', '£', 'J', 'Q', '0', '6', 't', 'Ï', '¢', 'Ì', '}', '/', 'u', '\x9d', '\x95', 'n', 'W', ';', 'D', '±', 'V', 'l', 'k', '\x89', 's', 'q', '+', 'v', 'P']


In [6]:
def get_onehot(x):
    # Take input a string and convert to one-hot encoding
    vec_size = len(c2i.keys())
    n_seq = len(x)
    data = np.zeros((1, n_seq, vec_size))
    
    # For each element in the list
    for ix in range(n_seq):
        curr_char = x[ix]
        oh_index = c2i[curr_char]
        # print ix, curr_char, oh_index
        data[:, ix, oh_index] = 1
    return data

print (get_onehot('this is my string').argmax(2))

[[115  18  28 133  26  28 133  26   3   7  26 133 115   4  28 124  83]]


In [7]:
for ix in ds.Tweet_Text[:10]:
    print (get_onehot(ix).shape)

(1, 123, 138)
(1, 129, 138)
(1, 136, 138)
(1, 138, 138)
(1, 130, 138)
(1, 99, 138)
(1, 140, 138)
(1, 85, 138)
(1, 142, 138)
(1, 142, 138)


In [8]:
class CharNN(nn.Module):
    def __init__(self, in_shape=None, out_shape=None, hidden_shape=None):
        super(CharNN, self).__init__()
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.hidden_shape = hidden_shape
        self.n_layers = 1
        
        self.rnn = nn.LSTM(
            input_size=self.in_shape,
            hidden_size=self.hidden_shape,
            num_layers=self.n_layers,
            batch_first=True
        )
        self.out = nn.Linear(self.hidden_shape, self.out_shape)
    
    def forward(self, x, h):
        r_out, h_state = self.rnn(x, h)
        
        outs = []
        for ix in range(r_out.size(1)):
            current_out = F.softmax(self.out(r_out[:, ix, :]))
            outs.append(current_out)
        return torch.stack(outs, dim=1), h_state
    
    def predict(self, char, h=None, top_k=None):
        if h is None:
            h = self.init_hidden(1, gpu=False)
        
        x = get_onehot(char)
        out, h = self.forward(torch.FloatTensor(x), h)
        
        p = out.data
        if top_k is None:
            top_ch = np.arange(self.out_shape)
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        return i2c[char], h
    
    def init_hidden(self, batch_size, gpu=False):
        if gpu:
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape).cuda()),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)).cuda())
        return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)),
                Variable(torch.zeros(self.n_layers, batch_size, self.hidden_shape)))

In [9]:
model = CharNN(in_shape=138, out_shape=138, hidden_shape=256)
#model.summary()
print (model)

CharNN(
  (rnn): LSTM(138, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=138, bias=True)
)


In [10]:
# Load the weights
model.load_state_dict(torch.load('/home/shubham/all_projects/CB/Summer_2018/data/checkpoints/text_gen/model_256h_epoch_38.ckpt'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/shubham/all_projects/CB/Summer_2018/data/checkpoints/text_gen/model_256h_epoch_38.ckpt'

In [11]:
# model.predict('a', top_k=20)[0]

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [13]:
# Set to train mode
# model.cuda()
model.train()
N = 5000

for epoch in range(2):
    total_loss = 0
    # For each sequence
    for qx in range(N):
        seqx = ds.Tweet_Text[qx]
        #print(seqx)
        print(len(seqx[:-1]))
        print(len(seqx[1:]))
        h_state = model.init_hidden(1)
        input_seq = seqx[:-1]
        target_seq = seqx[1:]
        
        x = Variable(torch.FloatTensor(get_onehot(input_seq)), requires_grad=True)# .cuda()
        y = Variable(torch.LongTensor(get_onehot(target_seq).argmax(2)))# .cuda()
        
        model.zero_grad()
        pred, h_state = model.forward(x, h_state)
        # print pred.squeeze().shape, y.shape
        loss = criterion(pred.squeeze(), y.squeeze())
        
        # optimizer.zero_grad()
        loss.backward()
        
        # gradient clipping to solve exploding/vanishing grads
        # clip = 5.0
        # nn.utils.clip_grad_norm(net.parameters(), clip)
        
        optimizer.step()
        total_loss += loss
        if qx%(N/5) == 0:
            print ('Loss: {} at Epoch: {} | Seq: {}'.format(loss, epoch, qx))
        
    print ("OverflowErrorerflowErrorall Average Loss: {} at Epoch: {}".format(total_loss / float(N), epoch))
    
    # Save model checkpoints
    if epoch % 10 == 0:
        torch.save(model.state_dict(), "./data/checkpoints/text_gen/model_256h_epoch_{}.ckpt".format(epoch))

122
122


C:\Users\hamuj\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loss: 4.927226543426514 at Epoch: 0 | Seq: 0
128
128
135
135
137
137
129
129
98
98
139
139
84
84
141
141
141
141
141
141
72
72
129
129
96
96
111
111
59
59
139
139
140
140
33
33
139
139
79
79
91
91
136
136
136
136
77
77
138
138
141
141
127
127
143
143
120
120
83
83
32
32
141
141
33
33
134
134
135
135
134
134
134
134
140
140
144
144
125
125
136
136
97
97
73
73
102
102
139
139
147
147
136
136
76
76
147
147
38
38
86
86
111
111
111
111
139
139
132
132
139
139
24
24
149
149
79
79
116
116
139
139
140
140
56
56
135
135
127
127
145
145
85
85
144
144
77
77
123
123
141
141
145
145
133
133
57
57
141
141
139
139
143
143
136
136
130
130
111
111
141
141
52
52
141
141
116
116
127
127
135
135
141
141
138
138
111
111
124
124
139
139
136
136
137
137
147
147
142
142
145
145
115
115
105
105
132
132
104
104
74
74
121
121
83
83
125
125
137
137
147
147
138
138
130
130
133
133
128
128
54
54
95
95
140
140
140
140
124
124
139
139
142
142
138
138
140
140
143
143
141
141
109
109
138
138
137
137
134
134
140
140
90


KeyboardInterrupt: 

In [21]:
sentence = 'o'
model.cpu()
h_s = model.init_hidden(1, gpu=False)
for ix in range(1000):
    char = sentence[-1]
    out, h = model.predict(char, h=h_s, top_k=100)
    h_s = (h[0].data, h[1].data)
    
    sentence += out
print sentence

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


or @Markeressoness @Markeresson @realDonaldTrump @Markething and an amappathing and an amarian and an amasting and an amarinate and an amater and an amarinate and an amas and an amasting and an amater and an amapart and an amarian and an amarian and an amarical and an amas and an amarinate and an amas and an amarian and an amas and an amasting and an amarian and an amarian and an amarian and an amasting and an amas and an amas and an amas and an amater and an amas and an amappare and an amas and an amarical and an amas and an amappard in an amaring and an amater and an amappare and an amas and an amarian and an amarinate and an amasting and an aman and an amater and an amas and an amater and an amarical and an amarian and an amarical and an amas and an amas and an amas and an amasting and an amarical and an amas and an amas and an amaricated and an amas and an amarian and an amas and an amater and an amas and an amas and an amasting and an amas and an amater and an amas and an amater a